In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\chest-cancer-classification'

In [3]:
#os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow"

In [4]:
import tensorflow as tf

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_path: Path
    trainig_data: Path
    all_params:  dict
    mlflow_url: str
    params_image_size: list
    params_batch_size: int

In [6]:
from chest_cancer.constants import *
from chest_cancer.utils.commen import read_yaml, create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        model_evaluation_config = ModelEvaluationConfig(
          model_path = "artifacts/model_training/model.keras",
          trainig_data = "artifacts/data_ingestion/chest-cancer-ct",
          mlflow_url = "https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow",
          all_params = self.params,
          params_image_size = self.params.IMAGE_SIZE,
          params_batch_size = self.params.BATCH_SIZE
        )

        return model_evaluation_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub
dagshub.init(repo_owner='ajuarjun528', repo_name='chest-cancer-classification', mlflow=True)

2025-01-22 22:27:52,190 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as ajuarjun528

2025-01-22 22:27:52,226 - INFO - helpers - Accessing as ajuarjun528
2025-01-22 22:27:55,799 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/repos/ajuarjun528/chest-cancer-classification "HTTP/1.1 200 OK"
2025-01-22 22:28:01,407 - INFO - _client - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ajuarjun528/chest-cancer-classification"

2025-01-22 22:28:01,421 - INFO - helpers - Initialized MLflow to track repo "ajuarjun528/chest-cancer-classification"


Repository ajuarjun528/chest-cancer-classification initialized!

2025-01-22 22:28:01,428 - INFO - helpers - Repository ajuarjun528/chest-cancer-classification initialized!


In [9]:
class Evaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.trainig_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url = urlparse(mlflow.get_tracking_uri()).scheme
    
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
    
            if tracking_url != "file":
                try:
                    # Ensure valid model registry parameters
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                except Exception as e:
                    print(f"Error registering model: {e}")
                    raise e
            else:
                mlflow.keras.log_model(self.model, "model")


In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

2025-01-22 22:28:01,609 - INFO - commen - yaml file: config\config.yaml loaded successfully
2025-01-22 22:28:01,618 - INFO - commen - yaml file: params.yaml loaded successfully
2025-01-22 22:28:01,622 - INFO - commen - Created directory at artifacts
Found 102 images belonging to 2 classes.


d:\chest-cancer-classification\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9510 - loss: 0.2238
2025-01-22 22:28:15,301 - INFO - commen - Saved json file at scores.json


2025/01/22 22:28:21 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/01/22 22:28:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/01/22 22:29:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


🏃 View run flawless-grub-588 at: https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow/#/experiments/0/runs/1f2c9c777e1e4629a33a46662cc82e15
🧪 View experiment at: https://dagshub.com/ajuarjun528/chest-cancer-classification.mlflow/#/experiments/0
